In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/citation_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/citation_sum


In [3]:
# works for BART and T5 only
#!pip3 install transformers==2.8.0

# For BART, T5, Pegasus
#!pip3 install transformers==3.5.0
!pip3 install -q git+https://github.com/huggingface/transformers

!pip3 install -q torch==1.7.0

!pip3 install -q sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 7.2 MB/s 
     |████████████████████████████████| 3.3 MB 69.5 MB/s 
     |████████████████████████████████| 895 kB 25.7 MB/s 
     |████████████████████████████████| 59 kB 9.2 MB/s 
     |████████████████████████████████| 776.7 MB 3.8 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
     |████████████████████████████████| 1.2 MB 6.8 MB/s 


In [4]:
# Checking out the GPU we have access to
!nvidia-smi

Sat Nov 20 15:44:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# Importing stock libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

#from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartTokenizer, BartForConditionalGeneration
#from transformers import PegasusTokenizer, PegasusForConditionalGeneration
#from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration

In [6]:
## Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [7]:
device

'cuda'

### Read arXiv training dataset

In [ ]:
import json

fileName = "train_data_50k.jsonl"
DATA_PATH = "arXiv_dataset/arxiv-dataset"

data = []
#with open(f"{DATA_PATH}/val.txt") as f:
with open(f"{DATA_PATH}/{fileName}") as f:
  for idx, line in enumerate(f):
    if idx > 2.5e4:
      break
    data.append(json.loads(line))
f.close()

In [ ]:
len(data)

25001

### Parse the article abstract and body (text)

In [ ]:
final_data = []

for article in data:
  article_abstract = "".join(article['abstract_text']).replace('<S>', '').replace('</S>', '')
  article_text = "".join(article['article_text'])

  final_data.append([article_text, article_abstract])

df = pd.DataFrame(final_data, columns=['article_text', 'article_abstract'])

In [ ]:
df.head()

,article_text,article_abstract
0,additive models @xcite provide an important fa...,additive models play an important role in sem...
1,the leptonic decays of a charged pseudoscalar ...,"we have studied the leptonic decay @xmath0 , ..."
2,the transport properties of nonlinear non - eq...,"in 84 , 258 ( 2000 ) , mateos conjectured tha..."
3,studies of laser beams propagating through tur...,the effect of a random phase diffuser on fluc...
4,the so - called `` nucleon spin crisis '' rais...,with a special intention of clarifying the un...


In [ ]:
df.article_text = 'summarize: ' + df.article_text

In [ ]:
df.head()

,article_text,article_abstract
0,summarize: additive models @xcite provide an i...,additive models play an important role in sem...
1,summarize: the leptonic decays of a charged ps...,"we have studied the leptonic decay @xmath0 , ..."
2,summarize: the transport properties of nonline...,"in 84 , 258 ( 2000 ) , mateos conjectured tha..."
3,summarize: studies of laser beams propagating ...,the effect of a random phase diffuser on fluc...
4,summarize: the so - called `` nucleon spin cri...,with a special intention of clarifying the un...


In [ ]:
df.shape

(25001, 2)

### Select which model to fine-tune

In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training

#tokenizer = T5Tokenizer.from_pretrained("t5-base")
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
#tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
#tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased-cnndm')

MAX_LEN = 512
SUMMARY_LEN = 150
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 2
LEARNING_RATE = 1e-4

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage 
# for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.article_abstract = self.data.article_abstract
        self.article_text = self.data.article_text

    def __len__(self):
        return len(self.article_abstract)

    def __getitem__(self, index):
        article_text = str(self.article_text[index])
        article_text = ' '.join(article_text.split())

        article_abstract = str(self.article_abstract[index])
        article_abstract = ' '.join(article_abstract.split())

        source = self.tokenizer.batch_encode_plus([article_text], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([article_abstract], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')


        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=42).reset_index(drop=True)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

FULL Dataset: (25001, 2)
TRAIN Dataset: (20001, 2)
TEST Dataset: (5000, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [8]:
#model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
#model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
#model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased-cnndm')

model = model.to(device)

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
def train(epoch):
  model.train()
  for _, data in enumerate(training_loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)

    loss = outputs[0]
    
    if _%500==0:
        print(f'Epoch: {epoch}, Loss:  {loss.item()}')
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
def _get_config_file(model_path, model_name):
    # Name of the file for storing hyperparameter details
    return os.path.join(model_path, model_name + ".config")

def _get_model_file(model_path, model_name):
    # Name of the file for storing network parameters
    return os.path.join(model_path, model_name + ".tar")

def load_model(model_path, model_name, net=None):
    """
    Loads a saved model from disk.

    Inputs:
        model_path - Path of the checkpoint directory
        model_name - Name of the model (str)
        net - (Optional) If given, the state dict is loaded into this model. Otherwise, a new model is created.
    """
    config_file, model_file = _get_config_file(model_path, model_name), _get_model_file(model_path, model_name)
    assert os.path.isfile(config_file), f"Could not find the config file \"{config_file}\". Are you sure this is the correct path and you have your model config stored here?"
    assert os.path.isfile(model_file), f"Could not find the model file \"{model_file}\". Are you sure this is the correct path and you have your model stored here?"
    with open(config_file, "r") as f:
        config_dict = json.load(f)
    if net is None:
        act_fn_name = config_dict["act_fn"].pop("name").lower()
        act_fn = act_fn_by_name[act_fn_name](**config_dict.pop("act_fn"))
        net = BaseNetwork(act_fn=act_fn, **config_dict)
    net.load_state_dict(torch.load(model_file, map_location=device))
    return net

def save_model(model, model_path, model_name):
    """
    Given a model, we save the state_dict and hyperparameters.

    Inputs:
        model - Network object to save parameters from
        model_path - Path of the checkpoint directory
        model_name - Name of the model (str)
    """
    config_dict = model.config
    os.makedirs(model_path, exist_ok=True)
    config_file, model_file = _get_config_file(model_path, model_name), _get_model_file(model_path, model_name)
    with open(config_file, "w") as f:
        json.dump(config_dict, f)
    torch.save(model.state_dict(), model_file)

### Call to the train method above and save the model

In [ ]:
for epoch in range(EPOCHS):
  train(epoch)

# save the trained pytorch model----change the named of the model in accordance with the baseline model currently being worked on
model_name = "arXiv-Prophetnet"    # changes with each baseline model or the proposed approach's name
model_path = f"arXiv-pytorch_models"
os.makedirs(model_path, exist_ok=True)

# Save entire model
# Specify a path
model_path = f"arXiv-pytorch_models/{model_name}.pt"
torch.save(model, model_path)


# If only want to save the model parameters only in state_dict()
#model_path = f"arXiv-pytorch_models/{model_name}.pth"
#torch.save(model.state_dict(), model_path)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  5.212104797363281
Epoch: 0, Loss:  6.897027969360352
Epoch: 0, Loss:  7.044528484344482
Epoch: 0, Loss:  6.9428839683532715
Epoch: 0, Loss:  6.924999713897705
Epoch: 0, Loss:  6.9309916496276855
Epoch: 0, Loss:  6.887115955352783
Epoch: 0, Loss:  6.847286701202393
Epoch: 0, Loss:  7.296646595001221
Epoch: 0, Loss:  6.902984619140625
Epoch: 0, Loss:  6.477074146270752
Epoch: 0, Loss:  6.611220836639404
Epoch: 0, Loss:  7.001718997955322
Epoch: 0, Loss:  7.083792686462402
Epoch: 0, Loss:  6.806636810302734
Epoch: 0, Loss:  6.99543571472168
Epoch: 0, Loss:  7.282900333404541
Epoch: 0, Loss:  6.689901828765869
Epoch: 0, Loss:  6.751784801483154
Epoch: 0, Loss:  6.486169338226318
Epoch: 0, Loss:  6.771632671356201
Epoch: 1, Loss:  6.964242458343506
Epoch: 1, Loss:  7.380588054656982
Epoch: 1, Loss:  6.84810733795166
Epoch: 1, Loss:  6.5313897132873535
Epoch: 1, Loss:  6.720719814300537
Epoch: 1, Loss:  6.415548324584961
Epoch: 1, Loss:  6.6963791847229
Epoch: 1, Loss:  7.22

In [ ]:
!pwd

/content/drive/My Drive/Colab Notebooks/apex-codes/citation_sum


In [ ]:
!pip3 install transformers==3.5.0

     |████████████████████████████████| 1.3 MB 8.9 MB/s 
     |████████████████████████████████| 1.1 MB 60.9 MB/s 
     |████████████████████████████████| 2.9 MB 61.6 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.5.2
    Uninstalling tokenizers-0.5.2:
      Successfully uninstalled tokenizers-0.5.2
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.96
    Uninstalling sentencepiece-0.1.96:
      Successfully uninstalled sentencepiece-0.1.96
  Attempting uninstall: transformers
    Found existing installation: transformers 2.8.0
    Uninstalling transformers-2.8.0:
      Successfully uninstalled transformers-2.8.0


In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import (T5Tokenizer, T5ForConditionalGeneration, 
                          BartTokenizer, BartForConditionalGeneration,
                          PegasusTokenizer, PegasusForConditionalGeneration)
 

In [ ]:
model_name = "arXiv-Prophetnet"
model_path = f"arXiv-pytorch_models"

# model path
#model_path = f"arXiv-pytorch_models/{model_name}.pt"
model_path = f"../citation_sum/arXiv-pytorch_models/{model_name}.pt"

model = torch.load(model_path)

In [ ]:
os.makedirs('Prophetnet-checkpoints')
model.save_pretrained('../citation_sum/Prophetnet-checkpoints/')

In [ ]:
model = ProphetNetForConditionalGeneration.from_pretrained("../citation_sum/Prophetnet-checkpoints/")
#tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

### Saving/Loading entire model to the file

In [ ]:
# save the trained pytorch model----change the named of the model in accordance with the baseline model currently being worked on
from torchvision import models
from torchsummary import summary


model_name = "arXiv-BART"
model_path = f"arXiv-pytorch_models"

# model path
#model_path = f"pubmed-pytorch_models/{model_name}.pth"
model_path = f"arXiv-pytorch_models/{model_name}.pt"


#model = model.load_state_dict(torch.load(model_path))
model = torch.load(model_path)

In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

### Experiment with generating summaries

In [ ]:
# First, read a text from the file

t5_prepared_Text = "summarize: " + preprocess_text
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=70,
                                    max_length=120,
                                    early_stopping=True)

final_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
print(final_summary)

In [ ]:
def writer(predictions, actuals):
    # output_path = r'/content/drive/My Drive/Colab Notebooks/'
    # pred_file = output_path+'preds.txt'
    # actual_file = output_path+'actuals.txt'
    # with open(pred_file, 'w') as f:
    #     for pred in predictions:
    #         f.write('%s\n' % pred)

    # with open(actual_file, 'w') as f:
    #     for actual in actuals:
    #         f.write('%s\n' % actual)
    my_dict = {
        'Actual Headline': actuals,
        'Prediction': predictions       
        }
    final_output = pd.DataFrame(my_dict)

    return final_output


In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = T5ForConditionalGeneration.from_pretrained('t5-small')
# input_ids = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")  # Batch size 1
# outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, lm_labels=input_ids)
# loss, prediction_scores = outputs[:2]

# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = T5ForConditionalGeneration.from_pretrained('t5-small')
# input_ids = tokenizer.encode_plus("summarize: Hello, my dog is cute", return_tensors="pt")  # Batch size 1
# outputs = model.generate(input_ids['input_ids'])
# pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in outputs]

In [ ]:
# print(f'The loss is: {loss}')
# print(f'The Prediction score is {prediction_scores}')
# print(f'The output is {outputs}')
# print(f'The final output is {pred}')